In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [27]:
class Unit:
  
  def __init__(self, data, _children=(), _op=''):
    self.data = data
    self.grad = 0
    self._backward = lambda:None
    self._prev = set(_children)
    self._op = _op

  def __repr__(self):
    return f"Unit(data={self.data})" 

  def __rmul__(self, other):
    return self * other

  def __add__(self, other):
    other = other if isinstance(other, Unit) else Unit(other)
    out = Unit(self.data + other.data, (self, other), '+')
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    return out

  def __neg__(self):
    return self * -1
  
  def __sub__(self, other):
    return self + (-other)
    
  def __mul__(self, other):
    other = other if isinstance(other, Unit) else Unit(other)
    out = Unit(self.data * other.data, (self, other), '*')
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    return out

  def __pow__(self, other):
    assert isinstance(other, (int, float)), 'only ints and floats'
    out = Unit(self.data**other, (self, ), f'**{other}')
    def _backward():
      self.grad += other * self.data ** (other -1) * out.grad
    out._backward = _backward
    return out

  def __truediv__(self, other):
    return self * other**-1

  def exp(self):
    x = self.data
    out = Unit(math.exp(x), (self, ), 'exp')
    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward
    return out

  def sin(self):
    x = self.data
    out = Unit(math.sin(x), (self,), 'sin')
    def _backward():
      self.grad += math.cos(out.data) * out.grad 
    out._backward = _backward
    return out

  def backward(self):
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [28]:
a = Unit(8.0)
b = Unit(2.0)

print(a - b)

Unit(data=6.0)


In [44]:
x1 = Unit(2.0)
x2 = Unit(0.0)

w1 = Unit(-3.0)
w2 = Unit(1.0)

b = Unit(6.8813735870195432)

x1w1 = x1*w1
x2w2 = x2*w2
x1w1x2w2 = x1w1+x2w2

n = x1w1x2w2 + b

e = (2*n).exp()
o = (e - 1)/(e + 1)

o.backward()

In [46]:
print(x1.grad)
print(x2.grad)
print(w1.grad)
print(w2.grad)

-1.5
0.5
1.0
0.0
